Следующие 2 ячейки выполнять не нужно. Они нужны лишь для работы в Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/My Drive/Colab Notebooks/data

/content/drive/My Drive/Colab Notebooks/data


Запускать можно код, начиная со следующей ячейки

Подключаем библиотеки, загружаем csv файл



In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
flags_descripts = ["country", "zone", "square", "population", "continent_by_square", "continent_by_population","language","religion",
                   "vertical_count","horizontal_count","colors_count","has_red","has_green","has_blue","has_gold","has_white",
                   "has_black","has_orange", "dominant_color"]

In [4]:
df_train = pd.read_csv('Flags.csv', names=flags_descripts, sep=";")
print(df_train.shape)

(194, 19)


In [5]:
print(df_train)

            country  zone  square  ...  has_black  has_orange  dominant_color
0       Afghanistan     1     648  ...          1           0           green
1           Albania     1      29  ...          1           0             red
2           Algeria     1    2388  ...          0           0           green
3    American-Samoa     3       0  ...          0           1            blue
4           Andorra     1       0  ...          0           0            gold
..              ...   ...     ...  ...        ...         ...             ...
189   Western-Samoa     3       3  ...          0           0             red
190      Yugoslavia     1     256  ...          0           0             red
191           Zaire     2     905  ...          0           1           green
192          Zambia     2     753  ...          1           1           green
193        Zimbabwe     2     391  ...          1           0           green

[194 rows x 19 columns]


In [6]:
name_scale = ["country", "zone", "language", "has_red", "has_green", "has_blue", "has_gold", "has_white", "has_black", "has_orange", "dominant_color"]
strong_scale = ["square", "population", "vertical_count", "horizontal_count","colors_count"]
order_scale = ["continent_by_square", "continent_by_population"]


Три функции для нахождения расстояния между признаками 2-х объектов

In [7]:
def strong_dist_2_obj(pos1, pos2, data) :
  return abs(data[pos1] - data[pos2]) / abs(data.max() - data.min())

In [8]:
def order_dist_2_obj(pos1, pos2, data) :
  res = 0.0
  for i in range(data.size) :
    if ((data[pos1] < data[i] and  data[pos2] > data[i]) or
        (data[pos1] > data[i] and  data[pos2] < data[i])) :
        res += 1.0
        continue
    if ((data[pos1] != data[i] and  data[pos2] == data[i]) or
        (data[pos1] == data[i] and  data[pos2] != data[i])) :
          res += 0.5
          continue
  return res / (data.size - 1)

In [9]:
def name_dist_2_obj(pos1, pos2, data) :
  res = 0.0 
  for i in range(data.size) :
    if ((data[pos1] == data[i] and data[pos2] != data[i]) or
        (data[pos1] != data[i] and  data[pos2] == data[i])) :
      res += 1.0
  return res / data.size

Функция нахождения расстояния между 2-мя объектами

In [10]:
def dist_between_2_obj(pos1, pos2, df_train):
  dist = 0.
  for i in df_train:
    if (i in name_scale):
      dist += math.pow(name_dist_2_obj(pos1, pos2, df_train[i]), 2)
    if (i in strong_scale):
      dist += math.pow(strong_dist_2_obj(pos1, pos2, df_train[i]), 2)
    if (i in order_scale):
      dist += math.pow(order_dist_2_obj(pos1, pos2, df_train[i]), 2)
  return math.sqrt(dist / df_train.count(axis='rows').size)

In [11]:
assert dist_between_2_obj(1, 1, df_train) == 0.0
print(dist_between_2_obj(0, 1, df_train))

0.4151742795460787


Три функции для вычисления расстояния между объектами с учётом типов шкал

In [12]:
def calculate_strong_scale_dists(np_strong_scale_array, dists):
  
  np_strong_scale_array = np_strong_scale_array[:,np.newaxis]
  
  for i in range(np_strong_scale_array.shape[0]):
     dists[i] = np.sum(np.abs(np_strong_scale_array - np_strong_scale_array[i]) / np.abs(np.max(np_strong_scale_array) - np.min(np_strong_scale_array)), axis = 1)  
  

In [13]:
def calculate_name_scale_dists(np_name_scale_array, dists):
  
  for i in range(np_name_scale_array.shape[0]):
    for j in range(np_name_scale_array.shape[0]):
      for k in range(np_name_scale_array.shape[0]):
        if (((np_name_scale_array[i] == np_name_scale_array[k]) and (np_name_scale_array[j] != np_name_scale_array[k]))
        or ((np_name_scale_array[i] != np_name_scale_array[k]) and (np_name_scale_array[j] == np_name_scale_array[k]))):
          dists[i][j] += 1.0
  dists /= np_name_scale_array.shape[0]

In [14]:
def calculate_order_scale_dists(np_order_scale_array, dists):
  for i in range(np_order_scale_array.shape[0]):
    for j in range(np_order_scale_array.shape[0]):
      for k in range(np_order_scale_array.shape[0]):

        if (((np_order_scale_array[i] > np_order_scale_array[k]) and (np_order_scale_array[j] < np_order_scale_array[k]))
        or ((np_order_scale_array[i] < np_order_scale_array[k]) and (np_order_scale_array[j] > np_order_scale_array[k]))):
          dists[i][j] += 1.0

        elif (((np_order_scale_array[i] != np_order_scale_array[k]) and (np_order_scale_array[j] == np_order_scale_array[k]))
        or ((np_order_scale_array[i] == np_order_scale_array[k]) and (np_order_scale_array[j] != np_order_scale_array[k]))):
          dists[i][j] += 0.5
  dists /= (np_order_scale_array.shape[0] - 1)

Вызываем данные функции и получаем кучу матриц расстояний, которые необходимо просуммировать для получения попарного расстояния между объектами

In [15]:
np_name_scale_dists = np.zeros((len(name_scale), df_train.shape[0], df_train.shape[0]), np.float64)
for i in name_scale:
  calculate_name_scale_dists(np.array(df_train[i]), np_name_scale_dists[name_scale.index(i)])

np_order_scale_dists = np.zeros((len(order_scale), df_train.shape[0], df_train.shape[0]), np.float64)
for i in order_scale:
   calculate_order_scale_dists(np.array(df_train[i]), np_order_scale_dists[order_scale.index(i)])

np_strong_scale_dists = np.zeros((len(strong_scale), df_train.shape[0], df_train.shape[0]), np.float64)
for i in strong_scale:
   calculate_strong_scale_dists(np.array(df_train[i]), np_strong_scale_dists[strong_scale.index(i)])


Проверяем основные свойства метрик

In [16]:
for j in range(df_train.shape[0]): 
  for k in range(df_train.shape[0]):
    for i in range(len(strong_scale)):
      if (j == k):
        assert np_strong_scale_dists[i][j][k] == 0.0
      else:
        assert np_strong_scale_dists[i][j][k] == np_strong_scale_dists[i][k][j]
    for i in range(len(name_scale)):
      if (j == k):
        assert np_name_scale_dists[i][j][k] == 0.0
      else:
        assert np_name_scale_dists[i][j][k] == np_name_scale_dists[i][k][j]
    for i in range(len(order_scale)):
      if (j == k):
        assert np_order_scale_dists[i][j][k] == 0.0
      else:
        assert np_order_scale_dists[i][j][k] == np_order_scale_dists[i][k][j]


Вычисление попарного расстояния между объектами

In [17]:
dists = np.zeros((df_train.shape[0], df_train.shape[0]), np.float64)

for i in range(len(name_scale)):
  dists += np.power(np_name_scale_dists[i], 2)

for i in range(len(order_scale)):
  dists += np.power(np_order_scale_dists[i], 2)

for i in range(len(strong_scale)):
  dists += np.power(np_strong_scale_dists[i], 2)

n = len(name_scale) + len(order_scale) + len(strong_scale)
dists = math.sqrt(1/n) * np.power(dists,0.5)
print (dists)

[[0.         0.42655102 0.37395208 ... 0.44475186 0.46668589 0.17883936]
 [0.42655102 0.         0.51571292 ... 0.47788556 0.49831116 0.43589684]
 [0.37395208 0.51571292 0.         ... 0.45214499 0.44477872 0.40788746]
 ...
 [0.44475186 0.47788556 0.45214499 ... 0.         0.36214091 0.42631721]
 [0.46668589 0.49831116 0.44477872 ... 0.36214091 0.         0.44912133]
 [0.17883936 0.43589684 0.40788746 ... 0.42631721 0.44912133 0.        ]]


Находим объект, суммарное расстояние до которого от всех остальных объектов максимально.

In [18]:
np_summary_dists = np.zeros(dists.shape[0], np.float64)
np_summary_dists = np.sum(dists, axis = 1)
max = np.max(np_summary_dists)
print(max)
print(np_summary_dists.tolist().index(max))

105.40089193430285
157


Функция, считающая расстояние между 2 порядковыми признаками

In [19]:
def features_dists(feature_name1, feature_name2, df_train) :
  ord_feat1 = df_train[feature_name1]
  ord_feat2 = df_train[feature_name2]
  size = df_train.shape[0]
  res = 0.0
  for i in range(size) : 
    for j in range(i, size) :
      if ((ord_feat1[i] > ord_feat1[j] and ord_feat2[i] < ord_feat2[j]) or
          (ord_feat1[i] < ord_feat1[j] and ord_feat2[i] > ord_feat2[j])):
        res += 1.0
      if ((ord_feat1[i] != ord_feat1[j] and ord_feat2[i] == ord_feat2[j]) or
          (ord_feat1[i] == ord_feat1[j] and ord_feat2[i] != ord_feat2[j])):
        res += 0.5
  return res / ((size - 1) * size / 2) 

In [20]:
print(features_dists("continent_by_square", "continent_by_population", df_train))

0.08973879600448693
